In [6]:
import pandas as pd
import numpy as np
from selenium import webdriver
from IPython.core.display import display, HTML

In [7]:
driver = webdriver.Chrome('C:\chromedriver.exe')

In [8]:
# display 설정
display(HTML("<style>.container { width:90% !important; }</style>"))

In [9]:
# 전부
keyword_list= []

for time in range(24):
    for day in range(1,32):
        if (day < 10) & (time < 10):
            driver.get('https://datalab.naver.com/keyword/realtimeList.naver?datetime=2020-05-0{}T0{}%3A00%3A00&entertainment=2&groupingLevel=0&marketing=2&news=2&sports=2'.format(day,time))
            keyword_list.append(driver.find_element_by_class_name('field_list').text.split("\n")[1::2])
        
        elif (day >= 10) & (time < 10):
            driver.get('https://datalab.naver.com/keyword/realtimeList.naver?datetime=2020-05-{}T0{}%3A00%3A00&entertainment=2&groupingLevel=0&marketing=2&news=2&sports=2'.format(day,time))
            keyword_list.append(driver.find_element_by_class_name('field_list').text.split("\n")[1::2])

        elif (day < 10) & (time >= 10):
            driver.get('https://datalab.naver.com/keyword/realtimeList.naver?datetime=2020-05-0{}T{}%3A00%3A00&entertainment=2&groupingLevel=0&marketing=2&news=2&sports=2'.format(day,time))
            keyword_list.append(driver.find_element_by_class_name('field_list').text.split("\n")[1::2])

        else:
            driver.get('https://datalab.naver.com/keyword/realtimeList.naver?datetime=2020-05-{}T{}%3A00%3A00&entertainment=2&groupingLevel=0&marketing=2&news=2&sports=2'.format(day,time))
            keyword_list.append(driver.find_element_by_class_name('field_list').text.split("\n")[1::2])

In [12]:
keyword_pd= pd.DataFrame(keyword_list)
keyword_pd.to_csv('./dataset/naver_May.csv', encoding= 'CP949')

In [13]:
# 정체 후
keyword_pd_preprocessed= pd.read_csv('./dataset/naver_MAY.csv', encoding='CP949')

In [15]:
import pandas as pd
import numpy as np

def get_hot_word(data):
    temp_df = data.copy()
    date_ran = list(data['DATE'].unique())
    final_df = pd.DataFrame(np.zeros((0,11)), columns = ['DATE',1,2,3,4,5,6,7,8,9,10])
    
    score = 2
    for i in range(1,21):
        i = str(i)
        temp_df[i] = temp_df[i].map(lambda x: x+'*:*{}'.format(round(score,1)))
        score -= 0.1
    
    for cal_date in date_ran:
        word_list = list()
        temp_dict = dict()
        for k in range(1,21):
            k = str(k)
            for i in temp_df[temp_df['DATE'] == cal_date][k]:
                x, y = i.split('*:*')
                if x in word_list:
                    sc = temp_dict[x]
                    temp_dict[x] = float(sc) + float(y)
                else:
                    temp_dict[x] = float(y)
                word_list.append(x)
        word_list = set(word_list)
        middle_df = pd.DataFrame(np.zeros((0,2)), columns = ['WORD','SCORE'])
        for word in word_list:
            middle_df = middle_df.append({'WORD':word, 'SCORE':float(temp_dict[word])}, ignore_index = True)
    
        step = middle_df.sort_values(by = ['SCORE'], axis = 0, ascending=False)[0:20]
        step = list(step['WORD'])

        appending_dict = {'DATE':cal_date, 1:step[0],2:step[1],3:step[2],4:step[3],5:step[4],6:step[5],7:step[6],8:step[7],9:step[8],10:step[9]}
        final_df = final_df.append(appending_dict, ignore_index = True)
    
    return final_df

In [16]:
hothot = get_hot_word(keyword_pd_preprocessed)
hothot.to_csv('./dataset/HOT_BY_DAY(MAY).csv')

In [17]:
hothot

,DATE,1,2,3,4,5,6,7,8,9,10
0,2020-05-01,라임병,홍진영,창원시청홈페이지,근로자의날 은행,근로자의날,밥블레스유2,창원시청,이진아,남양주시청,명현만 위대한
1,2020-05-02,강원도 산불,고성 산불,남양주시청,전복감태김밥,제습기,고성,강원도 고성,편스토랑,류리가람,강원도 고성 산불
2,2020-05-03,로또 909회,왕기춘,gp,북한,제습기,그것이 알고싶다,생활방역,박해준,부부의세계 와인,하연주
3,2020-05-04,긴급재난지원금 사용처,김유진pd,긴급재난지원금 조회 사이트,북한,어린이날,하연주,긴급재난지원금kr,도경완,이원일,용마랜드
4,2020-05-05,발로란트,이솔이,박성광,덴탈마스크,재난지원금 조회,어린이날 선물,긴급재난지원금 사용처,도경완,이원일,김유진pd
5,2020-05-06,덴탈마스크,오승은,이낙연,인분,재난지원금 조회,어린이 덴탈마스크,용인,정영진,긴급재난지원금kr,사회적 거리두기
6,2020-05-07,이태원,영혼수선공,덴탈마스크,핫펠트,이태원 코로나,남양유업,게이,용인 확진자,정준영,국산 덴탈마스크
7,2020-05-08,이태원 코로나,이태원,킹클럽,치과용 마스크,이태원 클럽 코로나,덴탈마스크,외모지상주의,국산 덴탈마스크,용인 66번 확진자,한복 교복
8,2020-05-09,이태원 킹클럽,이태원 클럽 코로나,이태원 코로나,킹클럽,청주 코로나,플레이어,송승헌,이혜성,이태원 클럽,라조장
9,2020-05-10,로또 910회,블랙수면방,정준원,폴킴,이태원 코로나,제주 코로나,일요일 약국,ufc,김민교,이태원 클럽 코로나
